# 1. Charger le dataset Boston de scikit-learn



In [2]:
from sklearn.datasets import load_boston
data = load_boston()
X, y = data.data, data.target

### Explorer les variables du dataset Boston

In [5]:
feat_names = data.feature_names
print(feat_names)

['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT']


CRIM - per capita crime rate by town

ZN - proportion of residential land zoned for lots over 25,000 sq.ft.

INDUS - proportion of non-retail business acres per town.

CHAS - Charles River dummy variable (1 if tract bounds river; 0 otherwise)

NOX - nitric oxides concentration (parts per 10 million)

RM - average number of rooms per dwelling

AGE - proportion of owner-occupied units built prior to 1940

DIS - weighted distances to five Boston employment centres

RAD - index of accessibility to radial highways

TAX - full-value property-tax rate per $10,000

PTRATIO - pupil-teacher ratio by town

B - 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town

LSTAT - % lower status of the population

MEDV - Median value of owner-occupied homes in $1000's

Exemple:

In [6]:
print(X[0])
print(y[0])

[6.320e-03 1.800e+01 2.310e+00 0.000e+00 5.380e-01 6.575e+00 6.520e+01
 4.090e+00 1.000e+00 2.960e+02 1.530e+01 3.969e+02 4.980e+00]
24.0


Séparer les données en train et test sets


In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)


# 2. Entraîner un arbre de régression pour prédire la valeur immobilière, tracer l'arbre, et l'importance de chacune des features de la base. Evaluer les performances en utilisant l'erreur quadratique moyenne (EQM).

In [16]:
#insérer le code ici


EQM : 44.644957955909064


 ####  Essayons d'utiliser la fonction de scikit-learn qui permet de plotter les arbres : on se représentera mieux les règles de l'arbres de décision.

In [17]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
plt.figure(figsize=(10,5))
plot_tree(reg)
plt.plot()

#### Regardons l'importance de chacune des features dans l'arbre de décision :

In [ ]:
plt.figure(figsize=(10,5))
plt.bar([i for i in range(len(reg.feature_importances_))], reg.feature_importances_, tick_label=feat_names)
plt.show()

# 3. Entraîner plusieurs arbres de régression en faisant varier le paramètre max_depth. Evaluer les performances avec l'EQM, sur train et test, et tracer la courbe de cette métrique en fonction de max_depth. Que remarquez-vous?


In [ ]:
#insérer le code ici














plt.figure()
plt.plot(max_depth_ls, mse_train, color='red', label='Train')
plt.plot(max_depth_ls, mse_test, color='blue', label='Test')
plt.title('MSE regarding max_depth')
plt.legend()
plt.show()

On remarque que si l'on augmente max_depth, les performances s'améliorent, mais stagnent vite.
On constate aussi la variabilité des prédictions sur les courbes tracées.


# 4. Entraîner plusieurs forêts aléatoires en faisant varier le paramètre max_depth. Evaluer les performances avec l'EQM sur train et test, et tracer la courbe de cette métrique en fonction de max_depth. Que remarquez-vous?

In [ ]:
#insérer le code ici












    
    
plt.figure()
plt.plot(max_depth_ls, mse_train, color='red', label='Train')
plt.plot(max_depth_ls, mse_test, color='blue', label='Test')
plt.title('MSE regarding max_depth')
plt.legend()
plt.show()

In [ ]:


plt.figure(figsize=(15,10))
plt.bar([i for i in range(len(rf.feature_importances_))], rf.feature_importances_, tick_label=feat_names)
plt.show()



# 5. Montrer que scikit-learn aggrège les prédictions de chaque arbre de la forêt en faisait la moyenne de celles-ci

In [ ]:
import numpy


rf = RandomForestRegressor(max_depth=20, random_state=0,
                             n_estimators=100)    
rf = rf.fit(X_train, y_train)


# Pour récupérer chacun des arbres (déjà entraîné) : rf. estimator

dest = numpy.zeros((X_test.shape[0], len(rf.estimators_)))
estimators = numpy.array(rf.estimators_).ravel()
for i, est in enumerate(estimators):
    pred = est.predict(X_test)
    dest[:, i] = pred

average = numpy.mean(dest, axis=1)
r2_score(y_test, average)


In [ ]:
y_pred = rf.predict(X_test)
r2_score(y_test, y_pred)

# 6. Au lieu d'aggréger les arbres par la moyenne de leur prédictions, on veut apprendre sur X_train des coefficients pour pondérer les prédictions de chacun des arbres. Remplacer les features de la base par les prédictions de chacun des arbres, puis entraîner une régression linéaire. Représenter les valeurs des coefficients ainsi trouvés. Que remarquez-vous?

Idée : aggréger les arbres par la moyenne donne le même poids à chacun des arbres, qu'ils soit bon ou pas. On peut essayer d'améliorer ça en trouvant des poids adaptés à chacun des arbres.

--> on fait une régression linéaire pour trouver ces coefficients de pondération.

--> chercher $a_1, ..., a_B$ t. q. $f_{RF}(x) = \sum_{i=1, ..., B} a_i f_i(x)$

In [28]:
#insérer le code ici





In [29]:
# Création de la nouvelle base : 
X_train2, X_test2 = new_features(rf, X_train), new_features(rf, X_test)

In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train2, y_train)

In [ ]:
# On printe les coeffs : 
plt.figure(figsize = (20,10))
plt.bar([i for i in range(len(rf.estimators_))], lr.coef_)
plt.title('Coefficients pour chacun des arbres')
plt.show()

In [ ]:
y_pred = lr.predict(X_test2)
print(mean_squared_error(y_test, y_pred))
print(r2_score(y_test, y_pred))

# 7. Faire de même en remplaçant la régression linéaire par une régression LASSO (régression linéaire pénalisée)

Dans le lasso, on borne la norme L1 du vecteur des coeffs : 
$min \frac{1}{N} \sum_{i=1}^N (y_i - xi^T \beta)^2 + \alpha  \sum_{j=1}^p |\beta_j|$

In [ ]:
#compléter le code
from sklearn.linear_model import Lasso





# On printe les coeffs : 
plt.figure(figsize = (20,10))
plt.bar([i for i in range(len(rf.estimators_))], lr_pen.coef_)
plt.title('Coefficients pour chacun des arbres')
plt.show()


In [ ]:


y_pred = lr_pen.predict(X_test2)
print(mean_squared_error(y_test, y_pred))
print(r2_score(y_test, y_pred))





C'est mieux, mais toujours moins bien que la moyenne.

--> choisir le paramètre $\alpha$
--> le faire varier et comparer les résultats

In [ ]:
alphas = [0.01 * i for i in range(100)] +[1 + 0.1 * i for i in range(100)]
mse_train = []
mse_test = []
r2_train = []
r2_test = []
for i in range(0, len(alphas)):
    alpha = alphas[i]
    lrs = Lasso(max_iter=20000, alpha=alpha)
    lrs.fit(X_train2, y_train)
    mse_train.append(mean_squared_error(y_train, lrs.predict(X_train2)))
    mse_test.append(mean_squared_error(y_test, lrs.predict(X_test2)))
    r2_train.append(r2_score(y_train, lrs.predict(X_train2)))
    r2_test.append(r2_score(y_test, lrs.predict(X_test2)))

In [ ]:
plt.figure()
plt.plot(alphas, mse_train, color='red', label='Train')
plt.plot(alphas, mse_test, color='blue', label='Test')
plt.title('EQM vs. max_depth')
plt.legend()
plt.show()
